# MI Linear Programming with Python (SCIP, Gurobi, C-Plex, GLPK, CBC, PuLP)


**An Example:** solve the following set of equations:

<center>$\large\max(x+y),\space$ subject to:


$-x+2y\le7$

$2x+y\le14$

$2x-y\le10$

$0\le x\le10$

$0\le y\le 10$</center>

#### Install Required Packages

In [1]:
# ! pip install -U ortools

In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()


✨🍰✨ Everything looks OK!


In [3]:
!conda install pyscipopt

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [4]:
from ortools.linear_solver import pywraplp

#### Create the Solver instance using CBC optimizer.

In [5]:
solver = pywraplp.Solver.CreateSolver('CBC')  # GLOP does not support integer variables

#### Enter the equations

In [6]:
x = solver.IntVar(0, 10, 'x')
y = solver.NumVar(0, 10, 'y')

solver.Add(-x + 2 * y <= 7.0)
solver.Add(2 * x + y <= 14.0)
solver.Add(2 * x - y <= 10.0)


<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7e5a10902cd0> >

#### Enter the objective function

In [7]:
solver.Maximize(x + y)
result = solver.Solve()

#### Get solution

In [8]:
if result == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('x =', x.solution_value())
    print('y =', y.solution_value())

Solution:
Objective value = 9.5
x = 4.0
y = 5.5


## SCIP (Solving Constraint Integer Programs)

In [9]:
from pyscipopt import Model

model = Model("example")

In [10]:
x = model.addVar(vtype="I", name="x")  # specify x as integer variable
y = model.addVar(vtype="C", name="y")

model.setObjective(x + y, sense="maximize")

In [11]:
model.addCons(-x + 2 * y <= 7)
model.addCons(2 * x + y <= 14)
model.addCons(2 * x - y <= 10)

c3

In [12]:
model.optimize()

solution = model.getBestSol()
print("Optimal Solution:")
print("x =", solution[x])
print("y =", solution[y])

Optimal Solution:
x = 4.0
y = 5.5


## Gurobi

In [13]:
!pip install gurobipy

In [14]:
! pip install pyomo

In [15]:
! pip install cplex

In [16]:
!conda install -c conda-forge glpk

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [17]:
import pyomo.environ as pyo
import gurobipy as gp
from pyomo.environ import *

In [18]:
model = pyo.ConcreteModel()

model.x = pyo.Var(domain=Integers,bounds=(0, 10))
model.y = pyo.Var(bounds=(0, 10))

x = model.x
y = model.y

model.C1 = pyo.Constraint(expr=-x + 2 * y <= 7)
model.C2 = pyo.Constraint(expr=2 * x + y <= 14)
model.C3 = pyo.Constraint(expr=2 * x - y <= 10)

model.obj = pyo.Objective(expr=x + y, sense=maximize)

In [19]:
opt = pyo.SolverFactory('gurobi')
opt.solve(model)

{'Problem': [{'Name': 'x1', 'Lower bound': 9.5, 'Upper bound': 9.5, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 2, 'Number of binary variables': 0, 'Number of integer variables': 1, 'Number of continuous variables': 1, 'Number of nonzeros': 6, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Return code': 0, 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': 0.0015270709991455078, 'Error rc': 0}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [20]:
model.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   4.0 :    10 : False : False : Integers
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   5.5 :    10 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : x + y

3 Constraint Declarations
    C1 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :  -Inf : - x + 2*y :   7.0 :   True
    C2 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x + y :  14.0 :   True
    C3 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x - y :  10.0 :   True

6 Declarations: x y C1 C2 C3 obj


In [21]:
x = pyo.value(model.x)
y = pyo.value(model.y)
print("Optimal Solution:")
print("x =", x)
print("y =", y)

Optimal Solution:
x = 4.0
y = 5.5


In [22]:
opt = pyo.SolverFactory('glpk')
opt.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 9.5, 'Upper bound': 9.5, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 2, 'Number of nonzeros': 6, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '3', 'Number of created subproblems': '3'}}, 'Error rc': 0, 'Time': 0.004616737365722656}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [23]:
model.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   4.0 :    10 : False : False : Integers
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   5.5 :    10 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : x + y

3 Constraint Declarations
    C1 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :  -Inf : - x + 2*y :   7.0 :   True
    C2 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x + y :  14.0 :   True
    C3 : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x - y :  10.0 :   True

6 Declarations: x y C1 C2 C3 obj


In [24]:
x = pyo.value(model.x)
y = pyo.value(model.y)
print("Optimal Solution:")
print("x =", x)
print("y =", y)

Optimal Solution:
x = 4.0
y = 5.5


## PuLP - Python Linear Programming

In [25]:
! pip install cython

In [26]:
! pip install pulp

In [27]:
import pulp as pl

In [28]:
model = pl.LpProblem("Simple_LP", pl.LpMaximize)

x = pl.LpVariable("x", lowBound=0,upBound=10,cat=pl.LpInteger)
y = pl.LpVariable("y", lowBound=0,upBound=10)

model += -x + 2 * y <= 7
model += 2 * x + y <= 14
model += 2 * x - y <= 10

model += x + y


In [29]:
status = model.solve()
print(pl.LpStatus[status])

Optimal


In [30]:
x = pl.value(x)
y = pl.value(y)
print("Optimal Solution:")
print("x =", x)
print("y =", y)

Optimal Solution:
x = 4.0
y = 5.5


# Exercise

Using Pyomo, find the optimum values of $x_i$ and $y$ to satisfy this Objective Function:   

<center>$\large\min\sum\limits_{i=1}^{5} x_i+y$, subject to:

$(a)\space\space\space\sum\limits_{i=1}^{5} x_i+y\le20$

$(b)\space\space\space\space x_i+y\ge15\space\forall i $

$(c)\space\space\space\sum\limits_{i=1}^{5} (i\cdot x_i)\ge10$

$(d)\space\space\space\space x_5+2y\ge30$

$(e)\space\space\space\space x_i,y\ge0$

$(f)\space\space\space\space x_i$ is integer $\forall i$</center>

#### Import packages

In [31]:
import time, numpy as np
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

#### Specify model and variables

In [32]:
model = pyo.ConcreteModel()

range_i = range(1,6)
model.x = pyo.Var(range_i, within=Integers, bounds=(0,None))
model.y = pyo.Var(bounds=(0,None))
x = model.x   #  shorthand for x
y = model.y   #  shorthand for y

#### Specify constraints

In [33]:
sum_x = sum(x[i] for i in range_i)

model.a = pyo.Constraint(expr =sum_x+y<=20)                           # constraint (a)
model.b = pyo.ConstraintList()
for i in range_i:
    model.b.add(expr = x[i]+y>=15)                                    # constraint (b)
model.c = pyo.Constraint(expr = sum([i*x[i] for i in range_i]) >= 10) # constraint (c)
model.d = pyo.Constraint(expr = x[5]+2*y >= 30)                       # constraint (d)

#### Specify objective

In [34]:
model.obj = pyo.Objective(expr = sum_x+y, sense=minimize)


#### Solve the system of equations

In [35]:
begin = time.time()
opt = SolverFactory('gurobi')
opt.solve(model)

deltaT = time.time() - begin

model.pprint()

2 Var Declarations
    x : Size=5, Index={1, 2, 3, 4, 5}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : Integers
          2 :     0 :  -0.0 :  None : False : False : Integers
          3 :     0 :  -0.0 :  None : False : False : Integers
          4 :     0 :  -0.0 :  None : False : False : Integers
          5 :     0 :   2.0 :  None : False : False : Integers
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  15.0 :  None : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[1] + x[2] + x[3] + x[4] + x[5] + y

4 Constraint Declarations
    a : Size=1, Index=None, Active=True
        Key  : Lower : Body                                 : Upper : Active
        None :  -Inf : x[1] + x[2] + x[3] + x[4] + x[5] + y :  20.0 :   True
    b : S

#### Output the results

In [36]:
print('Time =', np.round(deltaT,2))

for i in range(1,6):
    print('x[%i] = %i' % (i, pyo.value(x[i])))
print('y = %.2f' % pyo.value(y))
print('Obj = ', pyo.value(model.obj))

Time = 0.01
x[1] = 0
x[2] = 0
x[3] = 0
x[4] = 0
x[5] = 2
y = 15.00
Obj =  17.0
